In [4]:
import pandas_datareader as web
import pandas as pd
from yahoo_fin import stock_info as si
import datetime as dt

In [5]:
tickers = si.tickers_sp500()

In [7]:
start = dt.datetime.now() - dt.timedelta(days=365)
end = dt.datetime.now()

In [8]:
sp500_df = web.DataReader('^GSPC', 'yahoo', start, end)

In [11]:
sp500_df['Pct Change'] = sp500_df['Adj Close'].pct_change()

In [12]:
#이게 결국에는 곱해갖고 비슷해지는 머 그런개념인듯
sp500_return = (sp500_df['Pct Change'] + 1).cumprod()[-1]

In [13]:
return_list = []
final_df = pd.DataFrame(columns=['Ticker', 'Latest_Price', 'Score', 'PE_Ratio', 'PEG_Ratio', 'SMA_150', 'SMA_200', '52_Week_Low', '52_Week_High'])

In [19]:
for ticker in tickers:
    df = web.DataReader(ticker,'yahoo', start, end)
    df.to_csv(f'stock_data_{ticker}.csv')
    
    df['Pct Change'] = df['Adj Close'].pct_change()
    stock_return = (df['Pct Change'] +1).cumprod()[-1]
    
    returns_compared = round((stock_return / sp500_return), 2)
    return_list.append(returns_compared)
    
    counter +=1
    if counter == 10:
        break

In [24]:
best_performers = pd.DataFrame(list(zip(tickers, return_list)), columns=['Ticker', 'Returns Compared'])

In [64]:
best_performers = best_performers[:30]

In [65]:
best_performers['Score'] = best_performers['Returns Compared'].rank(pct=True)*100

C:\Users\ddh10\AppData\Local\Temp/ipykernel_2000/169822585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_performers['Score'] = best_performers['Returns Compared'].rank(pct=True)*100


In [66]:
#수익률 상위 70% 이상 스코어? 스코어는 보니까 높을수록 좋은듯
best_performers = best_performers[best_performers['Score'] >= best_performers['Score'].quantile(0.7)]

In [68]:
#수익률 상위 30% 이내 애들 중에, 150일, 200일 이동평균 . rolling은 뒤에 mean, std, var등 메서드를 붙여줘야 완성됨.
for ticker in best_performers['Ticker']:
    try:
        df = pd.read_csv(f'stock_data_{ticker}.csv', index_col=0)
        moving_averages = [150, 200]
        for ma in moving_averages:
            df['SMA_' + str(ma)] = round(df['Adj Close'].rolling(window=ma).mean(), 2)
        
        latest_price = df['Adj Close'][-1]
        pe_ratio = float(si.get_quote_table(ticker)['PE Ratio (TTM)'])
        peg_ratio = float(si.get_stats_valuation(ticker)[1][4])
        moving_average_150 = df['SMA_150'][-1]
        moving_average_200 = df['SMA_200'][-1]
        low_52week = round(min(df['Low'][-(52*5):], 2))
        high_52week = round(max(df['High'][-(52*5):], 2))
        #오~ for 문 돌리면서 ticker 검사, 똑같은거의 score만 가져오겟다
        score = round(best_performers[best_performers['Ticker'] == ticker]['Score'].tolist())
        
        # 아니 조건식 이게됨? ㅋㅋ
        condition_1 = latest_price > moving_average_150 > moving_average_200
        #52주 저가보다 1.3배높은가
        condition_2 = latest_price >= (1.3 * low_52week)
        condition_3 = latest_price >= (0.75 * high_52week)
        condition_4 = pe_ratio < 40
        condition_5 = peg_ratio < 2
        
        if  condition_4 and condition_5:
            final_df = final_df.append({'Ticker': ticker,
                                        'Latest_Price': latest_price,
                                        'Score': score,
                                        'PE_Ratio': pe_ratio,
                                        'Peg_Ratio': peg_ratio,
                                        'SMA_150': moving_average_150,
                                        'SMA_200': moving_average_200,
                                        '52_Week_Low': low_52week,
                                        '52_Week_High': high_52week                                       
                                       }, ignore_index=True)
         #ignore index, 열 이름 무시하고 정수 이름 부여
    except Exception as e:
        print(f"{e} for {ticker}")

The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). for ALB
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). for AMAT
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). for APA
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). for BBWI
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). for BKR
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). for CF
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). for CMA
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). for COF
The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all(). for COP


In [41]:
df['Adj Close'].rolling(window=150).mean()

Date
2020-10-09           NaN
2020-10-12           NaN
2020-10-13           NaN
2020-10-14           NaN
2020-10-15           NaN
                 ...    
2021-10-01    182.610831
2021-10-04    182.857976
2021-10-05    183.174979
2021-10-07    183.544352
2021-10-08    183.895022
Name: Adj Close, Length: 251, dtype: float64

In [45]:
si.get_quote_table('AAPL')

{'1y Target Est': 167.92,
 '52 Week Range': '107.32 - 157.26',
 'Ask': '142.84 x 2200',
 'Avg. Volume': 80405556.0,
 'Beta (5Y Monthly)': 1.22,
 'Bid': '142.80 x 1100',
 "Day's Range": '142.56 - 144.17',
 'EPS (TTM)': 5.11,
 'Earnings Date': 'Oct 27, 2021 - Nov 01, 2021',
 'Ex-Dividend Date': 'Aug 06, 2021',
 'Forward Dividend & Yield': '0.88 (0.61%)',
 'Market Cap': '2.385T',
 'Open': 144.03,
 'PE Ratio (TTM)': 27.98,
 'Previous Close': 143.29,
 'Quote Price': 142.89999389648438,
 'Volume': 58773155.0}

In [47]:
si.get_stats_valuation('AAPL')

,0,1
0,Market Cap (intraday) 5,2.36T
1,Enterprise Value 3,2.44T
2,Trailing P/E,27.98
3,Forward P/E 1,25.16
4,PEG Ratio (5 yr expected) 1,1.38
5,Price/Sales (ttm),6.80
6,Price/Book (mrq),36.81
7,Enterprise Value/Revenue 3,7.02
8,Enterprise Value/EBITDA 7,21.96


In [54]:
df = pd.read_csv(f'stock_data_AAPL.csv', index_col=0)

In [59]:
ticker = 'AAPL'
moving_averages = [150, 200]
for ma in moving_averages:
    df['SMA_' + str(ma)] = round(df['Adj Close'].rolling(window=ma).mean(), 2)

latest_price = df['Adj Close'][-1]
pe_ratio = float(si.get_quote_table(ticker)['PE Ratio (TTM)'])
peg_ratio = float(si.get_stats_valuation(ticker)[1][4])
moving_average_150 = df['SMA_150'][-1]
moving_average_200 = df['SMA_200'][-1]
low_52week = round(min(df['Low'][-(52*5):], 2))
high_52week = round(max(df['High'][-(52*5):], 2))
#오~ for 문 돌리면서 ticker 검사, 똑같은거의 score만 가져오겟다
score = round(best_performers[best_performers['Ticker'] == ticker]['Score'].tolist())

# 아니 조건식 이게됨? ㅋㅋ
condition_1 = latest_price > moving_average_150 > moving_average_200
#52주 저가보다 1.3배높은가
condition_2 = latest_price >= (1.3 * low_52week)
condition_3 = latest_price >= (0.75 * high_52week)
condition_4 = pe_ratio < 40
condition_5 = peg_ratio < 2

if condition_1 and condition_2 and condition_3 and condition_4 and condition_5:
    final_df = final_df.append({'Ticker': ticker,
                                'Latest_Price': latest_price,
                                'Score': score,
                                'PE_Ratio': pe_ratio,
                                'Peg_Ratio': peg_ratio,
                                'SMA_150': moving_average_150,
                                'SMA_200': moving_average_200,
                                '52_Week_Low': low_52week,
                                '52_Week_High': high_52week                                       
                               }, ignore_index=True)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [69]:
#내림차순
final_df.sort_values(by='Score', ascending=False)
pd.set_option('display.max_columns', 10)
print(final_df)
final_df.to_csv('final.csv')

Empty DataFrame
Columns: [Ticker, Latest_Price, Score, PE_Ratio, PEG_Ratio, SMA_150, SMA_200, 52_Week_Low, 52_Week_High]
Index: []
